# Module F: Numerical Integration

In [1]:
import numpy as np
import scipy.integrate as integrate

1. Write a function my_int_calc(f,f0,a,b,N,option), where f is a function object, a and b are scalars such that a < b, N is a positive integer, and option is the string ‘rect’, ‘trap’, or ‘simp’. Let x be an array starting at a, ending at b, containing N evenly spaced elements. The output argument, I, should be an approximation to the integral of f(x), with initial condition f0, computed according to the input argument, option.

In [2]:
def my_int_calc(f: callable, f0: float, a: float, b: float, N: int, option: str) -> float:
    """
    Integrate a function f(x) from a to b using N points.
    Options: 'rect', 'trap' or 'simp'

    :param f: Function object
    :param f0: Initial condition
    :param a: Lower bound
    :param b: Upper bound
    :param N: Number of points
    :param option: Integration method (rect, trap or simp)
    :return: Approximation of the integral
    """

    # Create an array of N evenly spaced points
    x = np.linspace(a, b, N)

    # Calculate the width
    h = (b - a) / (N - 1)

    # Calculate the y values for each x value
    y = f(x)

    match option:
        case 'rect':
            for i in range(1, N):
                f0 += y[i]

            return f0 * h

        case 'trap':
            for i in range(1, N-1):
                f0 += 2 * y[i]

            return (f0 + y[0] + y[N-1]) * h / 2

        case 'simp':
            f0 += (h/3) * (y[0] + 2*sum(y[:N-2:2]) + 4*sum(y[1:N-1:2]) + y[N-1])
            return f0

    return 0


In [3]:
# Test my_int_calc

_f_test = lambda x: x**2
_f0_test = 0
_a_test = 0
_b_test = 9
_N_test = 10

_f_int = my_int_calc(_f_test, _f0_test, _a_test, _b_test, _N_test, 'rect')
print("Rectangle method: ", _f_int)

_f_int = my_int_calc(_f_test, _f0_test, _a_test, _b_test, _N_test, 'trap')
print("Trapezoid method: ", _f_int)

_f_int = my_int_calc(_f_test, _f0_test, _a_test, _b_test, _N_test, 'simp')
print("Simpson method  : ", _f_int)

_f_int = integrate.quad(_f_test, _a_test, _b_test)
print("Actual          : ", _f_int[0])

Rectangle method:  285.0
Trapezoid method:  244.5
Simpson method  :  176.33333333333331
Actual          :  243.0


In [4]:
# More my_int_calc tests
print("\n")

_f_test = lambda x: x**2
_f_int = my_int_calc(_f_test, 0, 0, 1, 3, 'rect')
print('Computed: ', _f_int)
print('Expected: ', 0.625)

_f_int = my_int_calc(_f_test, 0, 0, 1, 3, 'trap')
print('Computed: ', _f_int)
print('Expected: ', 0.375)

_f_int = my_int_calc(_f_test, 0, 0, 1, 3, 'simp')
print('Computed: ', _f_int)
print('Expected: ', 0.3333333333333333)



Computed:  0.625
Expected:  0.625
Computed:  0.375
Expected:  0.375
Computed:  0.3333333333333333
Expected:  0.3333333333333333


2. Write a function my_poly_int(x,y), where x and y are one-dimensional arrays of the same size, and the elements of x are unique and in ascending order. The function my_poly_int should (1) compute the Lagrange polynomial going through all the points defined by x and y and (2) return an approximation to the area under the curve defined by x and y, I, defined as the analytic integral of the Lagrange interpolating polynomial.

In [5]:
def my_poly_int(x: np.array, y: np.array) -> float:
    """
    Integrate a function defined by x and y.

    :param x: Array of x values
    :param y: Array of y values
    :return: Approximation of the integral
    """

    # Define X
    X = np.linspace(x[0], x[-1], len(x)*10)

    # Calculate the Lagrange polynomial
    # Initialize output array with zeros of same shape as X
    Y = np.zeros(len(X))

    numer_func = lambda x_val, x_j: x_val - x_j
    denom_func = lambda x_i, x_j: x_i - x_j

    # Iterate through each X to Interp
    for _X_i, _X in enumerate(X):
        _Y = 0

        # Calculate each polynomial (Pi)
        for i, _x_i in enumerate(x):
            # Initialize P_i
            P_i = 1

            # Get y value for x
            y_i = y[i]

            # Iterate through each x data
            for j, _x_j in enumerate(x):
                if j != i:
                    P_i = P_i * (numer_func(_X, _x_j)) / (denom_func(_x_i, _x_j))

            # Iterate L(x) step
            _Y += y_i * P_i

        # Add to Y
        Y[_X_i] = _Y

    # Calculate the area under the curve
    return integrate.simps(Y, X)

In [6]:
# Test my_poly_int
# y = x**2
_x_test = np.array([0, 1, 3, 4, 5, 6, 8, 9])
_y_test = np.array([0, 1, 9, 16, 25, 36, 64, 81])

_f_int = my_poly_int(_x_test, _y_test)
print("Computed: ", _f_int)
print("Expected: ", 243.0)


print("\n")

# y = x**3 - x**2 + x
_x_test = np.array([0, 1, 3, 4, 5, 6, 8, 9])
_y_test = _x_test**3 - _x_test**2 + _x_test

_f_int = my_poly_int(_x_test, _y_test)
print("Computed: ", _f_int)
print("Expected: ", 1437.75)

Computed:  243.0002464308097
Expected:  243.0


Computed:  1437.7530803851216
Expected:  1437.75


3. When will my_poly_int work worse than the trapezoid method?

Answer: my_poly_int will work worse than the trapezoid method when the function is not smooth (i.e. the function is not a polynomial).